In [ ]:
import sys
import importlib
import pandas as pd
sys.path.append('../lib')

import autograd_convenience as co
import autograd_plot as plots
import story

importlib.reload(co)
importlib.reload(plots)
importlib.reload(story)

In [ ]:
df_file = pd.read_csv("../data/Totalrapport_02232022.csv").drop('Unnamed: 8', axis=1)

In [ ]:
df_file.Stønadsområde.value_counts()

In [ ]:
ytelse = "Alderspensjon"
df = co.prepare_df(df_file, ytelse)

In [ ]:
df.head(12)

In [ ]:
autogroups = ["År-måned", "Automatiseringsgrad"]
selvgroups = ["År-måned", "Selvbetjening"]

# Filtre som funker på df
_selvbetjent = df["Selvbetjening"] == "Selvbetjent"
_innland = df["Utenlandstilsnitt"] == "Nasjonal"
_auto = df["Automatiseringsgrad"] == "Automatisk"
_all = pd.Series([True for i in range(df.shape[0])])

In [ ]:
pd.Series([True for i in range(df.shape[0])])

In [ ]:
figs = {}

In [ ]:
df["Automatiseringsgrad"]

In [ ]:
if df['Automatiseringsgrad'].str.contains('Automatisk').any():
    figs["auto_total"] = plots.plot_automatisering1(co.sharpen_df(df, _all, autogroups))
    figs["auto_total"].show()

    auto_selv_list = [[co.sharpen_df(df, _selvbetjent, autogroups), co.sharpen_df(df, ~_selvbetjent, autogroups)]]
    titles = ['Selvbetjent', 'Ikke selvbetjent']
    figs["auto_selv"] = plots.plot_automatisering_subplots(auto_selv_list, titles)

    auto_utland_list = [[co.sharpen_df(df, _innland, autogroups), co.sharpen_df(df, ~_innland, autogroups)]]
    titles = ["Innland", "Utland"]
    figs["auto_utland"] = plots.plot_automatisering_subplots(auto_utland_list, titles)

In [ ]:
if df['Selvbetjening'].str.contains('Selvbetjent').any():

    figs["selv_total"] = plots.plot_selvbetjening1(co.sharpen_df(df, _all, selvgroups))

    selv_utland_list = [[co.sharpen_df(df, _innland, selvgroups), co.sharpen_df(df, ~_innland, selvgroups)]]
    titles = ["Innland", "Utland"]
    figs["selv_utland"] = plots.plot_selvbetjening_subplots(selv_utland_list, titles)

In [ ]:
for key, value in figs.items():
    print(key)
    value.show()

In [ ]:
innledning = f"Statistikk på automatiserings- og selvbetjeningsgrad for førstegangsbehandling av {ytelse} i pesys. Data er basert på rapporten PSAK120. Saker opprettet av batch er eksludert."
ds = story.make_datastory(ytelse, figs, innledning)

In [ ]:
#ds.publish(url="https://nada.ekstern.dev.nav.no/api") #dev
#ds.publish(url="https://nada.intern.nav.no/api") #prod
#ds.update(url="https://nada.ekstern.dev.nav.no/api", token='7f3503bd-f54c-4d56-b222-6ca461213527')
#ds.update(url="https://nada.intern.nav.no/api", token='77e7eb36-5a5b-4a9e-b6f9-ccee1c5db2e3')